<a href="https://colab.research.google.com/github/Meichen1029/660Final/blob/main/Data_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instructor: Alfonso Berumen
## Pepperdine University

#**Setting up the initial notebook**
####Import Python packages

In [ ]:
#install a new package
!pip install wquantiles

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#import pandas:
import pandas as pd
#import numpy:
import numpy as np
#matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.stats import trim_mean
from statsmodels import robust
import wquantiles

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats import power

import seaborn as sns

#several others
import scipy
import sklearn
import graphviz

In [ ]:
%load_ext rpy2.ipython

##Inventory & Sales data

In [ ]:
inventory = pd.read_csv('/content/Inventory Warehouse March 2023.csv',skiprows=4)

In [ ]:
 sales = pd.read_csv('/content/Sales Order.csv',skiprows=4)

In [ ]:
inventory.info()

In [ ]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15837 entries, 0 to 15836
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Product #     8529 non-null   object 
 1   Description   7316 non-null   object 
 2   SO #          7318 non-null   object 
 3   Customer PO#  221 non-null    object 
 4   Shipped       6019 non-null   object 
 5   Unnamed: 5    1579 non-null   object 
 6   Qty Shipped   5961 non-null   object 
 7   Price         7424 non-null   object 
 8   Unnamed: 8    1685 non-null   object 
 9   Unnamed: 9    48 non-null     float64
dtypes: float64(1), object(9)
memory usage: 1.2+ MB


###Remove unneeded rows

In [ ]:
first = inventory['Unnamed: 0'].value_counts()
print(first)

In [ ]:
first_s = sales['Product #'].value_counts()
print(first_s)

1.03959E+11                            7070
June 5, 2023 12:35:38 PM PDT            222
Giving Children Hope                    221
Customer Purchase History               221
Date Range: 01/01/2021 - 06/05/2022     221
                                       ... 
Good Neighbors USA                        1
Glendale Korean SDA                       1
Fusion Community Services                 1
Fullerton Unified School District         1
World Harvest Charities_C                 1
Name: Product #, Length: 116, dtype: int64


In [ ]:
sales2 = sales.loc[~((sales['Product #'] == 'June 5, 2023 12:35:38 PM PDT') |
(sales['Product #'] == 'Customer Purchase History') |
(sales['Product #'] == 'Giving Children Hope') |
(sales['Product #'] == 'Date Range: 01/01/2021 - 06/05/2022') |
(sales['Description'] == 'Description')|
(sales['Shipped'] == 'Totals:')|
(sales['Shipped'] == 'Grand Totals:')|
(sales['Customer PO#'] == 'Customer PO#'))]

In [ ]:
inventory2 = inventory.loc[~((inventory['Unnamed: 0'] == 'April 7, 2023 10:51:23 AM PDT') |
(inventory['Unnamed: 0'] == 'Inventory Valuation Summary') |
(inventory['Unnamed: 0'] == 'Giving Children Hope') |
(inventory['Unnamed: 0'] == 'Grand Total:') |
(inventory['Unnamed: 0'] == 'Main')|
(inventory['Unnamed: 13'] == 'Location Total:') |
(inventory[' '] == 'Location Group Total:') |
(inventory['Part #'] == 'Part #'))]

In [ ]:
sales2 = sales.loc[~((sales['Product #'] == 'June 5, 2023 12:35:38 PM PDT') |
(sales['Product #'] == 'Customer Purchase History') |
(sales['Product #'] == 'Giving Children Hope') |
(sales['Product #'] == 'Date Range: 01/01/2021 - 06/05/2022') |
(sales['Description'] == 'Description')|
(sales['Shipped'] == 'Totals:')|
(sales['Shipped'] == 'Grand Totals:')|
(sales['Customer PO#'] == 'Customer PO#'))]

In [ ]:
inventory2

,Unnamed: 0,Part #,Description,Unnamed: 3,Unnamed: 4,Asset Account,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Qty,Unnamed: 11,,Unnamed: 13,Unit Cost,Asset Value,Unnamed: 16
1,A101F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,103958891551,Protein / pl,NaN,NaN,Default,NaN,NaN,NaN,NaN,1,NaN,Pl,NaN,$ 0.00,$ 0.00,NaN
4,A106F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,103958891272,Electric Wheelchair,NaN,NaN,Default,NaN,NaN,NaN,NaN,2,NaN,ea,NaN,$ 800.00,"$ 1,600.00",NaN
7,A107F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,NaN,103958891556,Local Programs / pl,NaN,NaN,Default,NaN,NaN,NaN,NaN,1,NaN,Pl,NaN,$ 0.00,$ 0.00,NaN
3067,NaN,103958891561,Protein / lbs,NaN,NaN,Default,NaN,NaN,NaN,NaN,"2,267",NaN,lbs,NaN,$ 2.08,"$ 4,715.36",NaN
3068,NaN,103958891562,Vegetable / lbs,NaN,NaN,Default,NaN,NaN,NaN,NaN,"1,308",NaN,lbs,NaN,$ 1.67,"$ 2,184.36",NaN
3069,NaN,105000000023,Cream Soup,NaN,NaN,Default,NaN,NaN,NaN,NaN,539,NaN,lbs,NaN,$ 1.11,$ 598.29,NaN


In [ ]:
sales2

NameError: ignored

###Fill in missing values for location

In [ ]:
inventory2.info()

In [ ]:
sales2.info()

In [ ]:
first2 = inventory2['Unnamed: 0'].value_counts()
print(first2)

In [ ]:
first_s2 = sales2['Product #'].value_counts()
print(first_s2)

In [ ]:
inventory2

,Unnamed: 0,Part #,Description,Unnamed: 3,Unnamed: 4,Asset Account,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Qty,Unnamed: 11,,Unnamed: 13,Unit Cost,Asset Value,Unnamed: 16
1,A101F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,103958891551,Protein / pl,NaN,NaN,Default,NaN,NaN,NaN,NaN,1,NaN,Pl,NaN,$ 0.00,$ 0.00,NaN
4,A106F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,103958891272,Electric Wheelchair,NaN,NaN,Default,NaN,NaN,NaN,NaN,2,NaN,ea,NaN,$ 800.00,"$ 1,600.00",NaN
7,A107F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,NaN,103958891556,Local Programs / pl,NaN,NaN,Default,NaN,NaN,NaN,NaN,1,NaN,Pl,NaN,$ 0.00,$ 0.00,NaN
3067,NaN,103958891561,Protein / lbs,NaN,NaN,Default,NaN,NaN,NaN,NaN,"2,267",NaN,lbs,NaN,$ 2.08,"$ 4,715.36",NaN
3068,NaN,103958891562,Vegetable / lbs,NaN,NaN,Default,NaN,NaN,NaN,NaN,"1,308",NaN,lbs,NaN,$ 1.67,"$ 2,184.36",NaN
3069,NaN,105000000023,Cream Soup,NaN,NaN,Default,NaN,NaN,NaN,NaN,539,NaN,lbs,NaN,$ 1.11,$ 598.29,NaN


In [ ]:
sales2

,Product #,Description,SO #,Customer PO#,Shipped,Unnamed: 5,Qty Shipped,Price,Unnamed: 8,Unnamed: 9
0,A.C. Green Youth Foundation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.03959E+11,MF Raisins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,167,NaN,8/20/2021,NaN,"1,512 lbs",$0.00,NaN,NaN
3,1.03959E+11,Rice Crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,445,NaN,10/14/2021,NaN,765 lbs,$25.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
15829,1.03959E+11,Furniture / pl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15830,NaN,NaN,1252,NaN,NaN,3/10/2022,NaN,1 Pl,$100.00,NaN
15831,4214,Payment for Discrepency on #331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15832,NaN,NaN,613,NaN,NaN,11/4/2021,NaN,1 ea,$400.00,NaN


In [ ]:
inventory2.reset_index(drop=True)
cols = ['Unnamed: 0']
inventory2.loc[:,cols] = inventory2.loc[:,cols].ffill()

<ipython-input-65-a494826ea169>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  inventory2.loc[:,cols] = inventory2.loc[:,cols].ffill()


In [ ]:
sales2.reset_index(drop=True)
cols_s1 = ['Product #']
cols_s2 = ['Description']
sales2.loc[:,cols_s1] = sales2.loc[:,cols_s1].ffill()
sales2.loc[:,cols_s2] = sales2.loc[:,cols_s2].ffill()

<ipython-input-44-32d3c4ecf0d6>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales2.loc[:,cols_s1] = sales2.loc[:,cols_s1].ffill()
<ipython-input-44-32d3c4ecf0d6>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales2.loc[:,cols_s2] = sales2.loc[:,cols_s2].ffill()


In [ ]:
inventory2

,Unnamed: 0,Part #,Description,Unnamed: 3,Unnamed: 4,Asset Account,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Qty,Unnamed: 11,,Unnamed: 13,Unit Cost,Asset Value,Unnamed: 16
1,A101F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A101F,103958891551,Protein / pl,NaN,NaN,Default,NaN,NaN,NaN,NaN,1,NaN,Pl,NaN,$ 0.00,$ 0.00,NaN
4,A106F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,A106F,103958891272,Electric Wheelchair,NaN,NaN,Default,NaN,NaN,NaN,NaN,2,NaN,ea,NaN,$ 800.00,"$ 1,600.00",NaN
7,A107F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066,,103958891556,Local Programs / pl,NaN,NaN,Default,NaN,NaN,NaN,NaN,1,NaN,Pl,NaN,$ 0.00,$ 0.00,NaN
3067,,103958891561,Protein / lbs,NaN,NaN,Default,NaN,NaN,NaN,NaN,"2,267",NaN,lbs,NaN,$ 2.08,"$ 4,715.36",NaN
3068,,103958891562,Vegetable / lbs,NaN,NaN,Default,NaN,NaN,NaN,NaN,"1,308",NaN,lbs,NaN,$ 1.67,"$ 2,184.36",NaN
3069,,105000000023,Cream Soup,NaN,NaN,Default,NaN,NaN,NaN,NaN,539,NaN,lbs,NaN,$ 1.11,$ 598.29,NaN


In [ ]:
sales2

,Product #,Description,SO #,Customer PO#,Shipped,Unnamed: 5,Qty Shipped,Price,Unnamed: 8,Unnamed: 9
0,A.C. Green Youth Foundation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.03959E+11,MF Raisins,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1.03959E+11,MF Raisins,167,NaN,8/20/2021,NaN,"1,512 lbs",$0.00,NaN,NaN
3,1.03959E+11,Rice Crackers,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.03959E+11,Rice Crackers,445,NaN,10/14/2021,NaN,765 lbs,$25.00,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
15829,1.03959E+11,Furniture / pl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15830,1.03959E+11,Furniture / pl,1252,NaN,NaN,3/10/2022,NaN,1 Pl,$100.00,NaN
15831,4214,Payment for Discrepency on #331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15832,4214,Payment for Discrepency on #331,613,NaN,NaN,11/4/2021,NaN,1 ea,$400.00,NaN


In [ ]:
inventory3 = inventory2

In [ ]:
sales3=sales2

In [ ]:
inventory3 = inventory3.rename(columns={"Unnamed: 0": "Main"," ":"Units"})

In [ ]:
inventory3 = inventory3.drop(['Unnamed: 3',
                 'Unnamed: 4',
                 'Unnamed: 6',
                 'Unnamed: 7',
                 'Unnamed: 8',
                 'Unnamed: 9',
                 'Unnamed: 11',
                 'Unnamed: 13',
                 'Unnamed: 16'],
                axis=1)

In [ ]:
sales3 = sales3.drop(['Unnamed: 9'],axis=1)

In [ ]:
inventory3.head(10)

In [ ]:
sales3

Keep remaining rows

In [ ]:
inventory4 = inventory3[inventory3['Part #'].notna()]

In [ ]:
sales4 = sales3[sales3['SO #'].notna()]

In [ ]:
inventory4.head(5)

In [ ]:
# Replace on single column
inventory4['Location'] = inventory4['Main'].replace(" ",np.nan,regex = True)
print(inventory4)

In [ ]:
inventory4.reset_index(drop=True)
cols = ['Location']
inventory4.loc[:,cols] = inventory4.loc[:,cols].ffill()

In [ ]:
sales4.drop(['Customer PO#'], axis=1, inplace=True)
sales4['Shipped'] = sales4['Shipped'].fillna(sales4['Unnamed: 5'])
sales4.drop(['Unnamed: 5'], axis=1, inplace=True)


In [ ]:
missing_rows = sales4['Qty Shipped'].isna()
sales4.loc[missing_rows, 'Qty Shipped'] = sales4.loc[missing_rows, 'Price'].shift(-1)
sales4.loc[missing_rows, 'Price'] = sales4.loc[missing_rows, 'Unnamed: 8']
sales4.drop('Unnamed: 8', axis=1, inplace=True)

In [ ]:
inventory4.head(30)

In [ ]:
print(sales4)

         Product #                      Description  SO #     Shipped  \
2      1.03959E+11                       MF Raisins   167   8/20/2021   
4      1.03959E+11                    Rice Crackers   445  10/14/2021   
6      1.03959E+11                       Mixed Food   445  10/14/2021   
8      1.03959E+11            Play Doh Jungle Train    54    8/3/2021   
10     1.03959E+11      Mixed Young Children's Toys    54    8/3/2021   
...            ...                              ...   ...         ...   
15824  1.03959E+11                   Furniture / pl   546   11/4/2021   
15826  1.03959E+11                   Furniture / pl   864  12/28/2021   
15828  1.03959E+11                   Furniture / pl   864  12/28/2021   
15830  1.03959E+11                   Furniture / pl  1252   3/10/2022   
15832         4214  Payment for Discrepency on #331   613   11/4/2021   

      Qty Shipped    Price  
2       1,512 lbs   $0.00   
4         765 lbs  $25.00   
6          80 lbs  $25.00   
8      

In [ ]:
inventory4 = inventory4.drop(['Main'],
                axis=1)

In [ ]:
inventory4

In [ ]:
sales4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7097 entries, 2 to 15832
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Product #    7097 non-null   object
 1   Description  7097 non-null   object
 2   SO #         7097 non-null   object
 3   Shipped      7097 non-null   object
 4   Qty Shipped  7096 non-null   object
 5   Price        7097 non-null   object
dtypes: object(6)
memory usage: 388.1+ KB


Save file

In [ ]:
from google.colab import files
inventory4.to_csv('inventory_march_2023_clean.csv')
files.download('inventory_march_2023_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
sales4.to_csv('Sales_Order_clean.csv')
files.download('Sales_Order_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>